In [10]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os, getpass
from langchain.memory import ConversationBufferMemory
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.callbacks import StdOutCallbackHandler
import gradio as gr
from agents import  RAGAgentLangChainOpenAI

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [3]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/var/folders/dk/8plcwpv53gg0_dg9fkx_rgr00000gn/T/ipykernel_62145/1046491335.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [7]:
embedding = OpenAIEmbeddings()
vectorstore = Chroma(persist_directory=f"../db/rag_db", embedding_function=embedding)

In [8]:
llm_openai = ChatOpenAI(temperature=0.7 , model="gpt-4o-mini")

In [9]:
custom_prompt = ChatPromptTemplate.from_template(
"""Use the information below to answer the user's question.
If you can't find a suitable answer, just say you don't know.
Don't give wrong or incorrect answers.

{context}

Câu hỏi: {question}
"""
)

In [11]:
ragLangChainOpenAI = RAGAgentLangChainOpenAI(memory=memory , vector_store=vectorstore, prompt_template=custom_prompt, llm=llm_openai)

In [12]:
def chat(question, history):
    result_conversation = ragLangChainOpenAI.invoke(question)
    return result_conversation["answer"]


view = gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
